In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
from PIL import Image

In [2]:
# definir le chemin du dataset
DATASET_FOLDER = "/home/nathan/id_shoes_pictures/dataset/images_vgg/"

In [ ]:
# # create folders to store images for classification 
# for data in ["train", "val", "test"]:
#     for classe in range(21):
#         path_folder = os.path.join(FOLDER_DIR, data, str(classe))
#         os.makedirs(path_folder, exist_ok=True)
#     for image in os.listdir(DATA_DIR+data):
#         path_src = os.path.join(DATA_DIR, data, image)
#         path_dest = os.path.join(FOLDER_DIR, data, image.split("_")[0])
#         shutil.copy(path_src, path_dest)

In [3]:
# definir un simple transformer pour convertir les images en tensor et aussi les normaliser
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# créer nos datasets train et val
train_data = datasets.ImageFolder(DATASET_FOLDER+"train", data_transforms["train"])
val_data = datasets.ImageFolder(DATASET_FOLDER+"val", data_transforms["val"])

In [ ]:
# create dataloader to feed our network with batchsize 20. For windows users, Olé :) , set num_workers to 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True, num_workers=4)

val_loader = torch.utils.data.DataLoader(val_data, batch_size=20, shuffle=False, num_workers=4)

In [4]:
# importer le modèle vgg16, freezer toutes les couches et remplacer la dernière couche mais avec des paramètres qui seront mis à jour
net = models.vgg16(pretrained=True)
for param in net.parameters():
    param.requires_grad = False
net.classifier[6] = nn.Linear(4096,21)

In [5]:
# utiliser le gpu s'il y'en a
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# utiliser la cross entropy comme fonction de perte
criterion = nn.CrossEntropyLoss()

# utiliser la descente de gradient stochastique et mettre uniquement à jour les parametres de la couche
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# train step
epochs = 50
best_acc = 0.0

for epoch in range(epochs):
    
    net.train()

    running_loss = 0.0
    for i, data in enumerate(train_loader):

        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 20 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, (i+1)*20, running_loss))
            
        running_loss = 0.0
        
    # we evaluate our model precision on val_dataset
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print('Accuracy of the network on the %d val images: %d %%' % (total, acc))
    
    torch.save(net.state_dict(), "/home/nathan/code/Cours Stefano Millaci/model_weights/id_shoes_vgg16_weights_last.pt")
    if acc > best_acc:
        best_acc = acc
        print(epoch+1)
        torch.save(net.state_dict(), "/home/nathan/code/Cours Stefano Millaci/model_weights/id_shoes_vgg16_weights_best.pt")

print('Finished Training')

In [9]:
# index de nos classes
class_names = {   
    "0":"AQS100",
    "1":"AQS120",
    "2":"AQUADOTS",
    "3":"AQUAFUN",
    "4":"BOCAGE 1",
    "5":"SAILING100",
    "6":"TBS 1",
    "7":"TBS 10",
    "8":"TBS 11",
    "9":"TBS 12",
    "10":"TBS 13",
    "11":"TBS 14",
    "12":"TBS 15",
    "13":"TBS 2",
    "14":"TBS 3",
    "15":"TBS 4",
    "16":"TBS 5",
    "17":"TBS 6",
    "18":"TBS 7",
    "19":"TBS 8",
    "20":"TBS 9"
}

#class_names = train_data.classes

In [ ]:
# sauvegarder les poids de notre modèles
#!mkdir /content/vgg16_model_weigths
torch.save(net.state_dict(), "/content/vgg16_model_weigths/id_shoes_weights_5_epochs_kfold_5.pt")

In [6]:
# pour charger les poids de notre modèle
net.load_state_dict(torch.load("/home/nathan/code/Cours Stefano Millaci/model_weights/id_shoes_vgg16_weights_best.pt"))

<All keys matched successfully>

In [10]:
# definir le dataset de test et le dataloader de test. Sur Windows mettre num_workers à 0
test_data = datasets.ImageFolder(DATASET_FOLDER+"test", data_transforms["val"])
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, num_workers=4,shuffle=True)

In [11]:
# code pour calculer la précision sur les données de test
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(test_loader):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Afficher la précision
        print('La précision est %d %%' % (100.0 * correct / total))

NameError: name 'epoch' is not defined

In [ ]:
# code pour calculer la précision sur les données de test
correct = 0
total = 0
with torch.no_grad():
    net.eval()
    for i, data in enumerate(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Afficher la précision
    print('La précision est %d %%' % (100.0 * correct / total))

In [ ]:
test_data

In [14]:
image = Image.open("/home/nathan/id_shoes_pictures/dataset/images_vgg/test/AQS100/20210211_154538.jpg")
image = data_transforms["val"](image).unsqueeze(0)
image = image.to(device)
net.eval()
output = net(image)
_, predicted = torch.max(output.data, 1)
display(output, predicted)

tensor([[18.0237, 12.8471,  6.2282,  1.7952,  0.2237,  1.5533, -0.8606, -5.5846,
         -5.4529, -0.5034, -2.8410,  2.7344,  2.4049,  3.1068, -6.5865, -5.1063,
         -0.3282, -5.3743, -8.0533, -2.1314, -4.4783]], device='cuda:0',
       grad_fn=<AddmmBackward>)

tensor([0], device='cuda:0')

In [ ]:
class_names[predicted.item()]

In [ ]:
# code pour calculer la précision sur chacune de nos classes (21 classes)
for epoch in range(5,51,5):
    net.load_state_dict(torch.load("/home/nathan/code/Cours Stefano Millaci/model_weights/id_shoes_weights_{}_epochs.pt".format(epoch)))
    class_correct = list(0. for i in range(21))
    class_total = list(0. for i in range(21))
    with torch.no_grad():
        net.eval()
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            # calculer le nombre de prédictions correctes pour la taille de batch 12
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print("Pour {} épochs".format(epoch))
    for i in range(21):
        print('Accuracy of %5s : %2d %%' % (class_names[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
# code pour calculer la précision sur chacune de nos classes (21 classes)
class_correct = list(0. for i in range(21))
class_total = list(0. for i in range(21))
with torch.no_grad():
    net.eval()
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        # calculer le nombre de prédictions correctes pour la taille de batch 12
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

            
for i in range(21):
    print('Accuracy of %5s : %2d %%' % (
        class_names[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
train_data.classes